In [9]:
pip install pdfminer.six pdfplumber pdf2image pytesseract pillow pdfminer PyPDF2 python-dateutil pytesseract pandas seaborn nltk sentence-transformers 

Note: you may need to restart the kernel to use updated packages.


In [10]:
Извлечение текста из pdf

SyntaxError: invalid syntax (3222260561.py, line 1)

In [5]:
import PyPDF2
# Для анализа структуры PDF и извлечения текста
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# Для извлечения текста из таблиц в PDF
import pdfplumber
# Для извлечения изображений из PDF
from PIL import Image
from pdf2image import convert_from_path
# Для выполнения OCR, чтобы извлекать тексты из изображений 
import pytesseract 
# Для удаления дополнительно созданных файлов
import os

import csv

# Создаём функцию для извлечения текста
def text_extraction(element):
    # Извлекаем текст из вложенного текстового элемента
    line_text = element.get_text()
    
    # Находим форматы текста
    # Инициализируем список со всеми форматами, встречающимися в строке текста
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Итеративно обходим каждый символ в строке текста
            for character in text_line:
                if isinstance(character, LTChar):
                    # Добавляем к символу название шрифта
                    line_formats.append(character.fontname)
                    # Добавляем к символу размер шрифта
                    line_formats.append(character.size)
    # Находим уникальные размеры и названия шрифтов в строке
    format_per_line = list(set(line_formats))
    
    # Возвращаем кортеж с текстом в каждой строке вместе с его форматом
    return (line_text, format_per_line)
  
# Создаём функцию для вырезания элементов изображений из PDF
def crop_image(element, pageObj):
    # Получаем координаты для вырезания изображения из PDF
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1] 
    # Обрезаем страницу по координатам (left, bottom, right, top)
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    # Сохраняем обрезанную страницу в новый PDF
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    # Сохраняем обрезанный PDF в новый файл
    with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

# Создаём функцию для преобразования PDF в изображения
def convert_to_images(input_file,):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = "PDF_image.png"
    image.save(output_file, "PNG")

# Создаём функцию для считывания текста из изображений
def image_to_text(image_path):
    # Считываем изображение
    img = Image.open(image_path)
    # Извлекаем текст из изображения
    text = pytesseract.image_to_string(img) #,lang='rus'
    return text
  
# Извлечение таблиц из страницы
def extract_table(pdf_path, page_num, table_num):
    # Открываем файл pdf
    pdf = pdfplumber.open(pdf_path)
    # Находим исследуемую страницу
    table_page = pdf.pages[page_num]
    # Извлекаем соответствующую таблицу
    table = table_page.extract_tables()[table_num]
    return table

# Преобразуем таблицу в соответствующий формат
def table_converter(table):
    table_string = ''
    # Итеративно обходим каждую строку в таблице
    for row_num in range(len(table)):
        row = table[row_num]
        # Удаляем разрыв строки из текста с переносом
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Преобразуем таблицу в строку
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Удаляем последний разрыв строки
    table_string = table_string[:-1]
    return table_string
  
pdf_path = 'instrukciya-po-ehkspluatacii-v8.pdf'

output_folder = '../data/extracted_files/doctorexplaindocument/'

# создаём объект файла PDF
pdfFileObj = open(pdf_path, 'rb')
# создаём объект считывателя PDF
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Создаём словарь для извлечения текста из каждого изображения
text_per_page = {}
# Извлекаем страницы из PDF
for pagenum, page in enumerate(extract_pages(pdf_path)):
    
    # Инициализируем переменные, необходимые для извлечения текста со страницы
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Инициализируем количество исследованных таблиц
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Открываем файл pdf
    pdf = pdfplumber.open(pdf_path)
    # Находим исследуемую страницу
    page_tables = pdf.pages[pagenum]
    # Находим количество таблиц на странице
    tables = page_tables.find_tables()


    # Находим все элементы
    page_elements = [(element.y1, element) for element in page._objs]
    # Сортируем все элементы по порядку нахождения на странице
    page_elements.sort(key=lambda a: a[0], reverse=True)

    # Находим элементы, составляющие страницу
    for i,component in enumerate(page_elements):
        # Извлекаем положение верхнего края элемента в PDF
        pos= component[0]
        # Извлекаем элемент структуры страницы
        element = component[1]
        
        # Проверяем, является ли элемент текстовым
        if isinstance(element, LTTextContainer):
            # Проверяем, находится ли текст в таблице
            if table_extraction_flag == False:
                # Используем функцию извлечения текста и формата для каждого текстового элемента
                (line_text, format_per_line) = text_extraction(element)
                # Добавляем текст каждой строки к тексту страницы
                page_text.append(line_text)
                # Добавляем формат каждой строки, содержащей текст
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Пропускаем текст, находящийся в таблице
                pass

        # Проверяем элементы на наличие изображений
        if isinstance(element, LTFigure):
            # Вырезаем изображение из PDF
            crop_image(element, pageObj)
            # Преобразуем обрезанный pdf в изображение
            convert_to_images('cropped_image.pdf')
            # Извлекаем текст из изображения
            image_text = image_to_text('PDF_image.png')
            text_from_images.append(image_text)
            page_content.append(image_text)
            # Добавляем условное обозначение в списки текста и формата
            page_text.append('image')
            line_format.append('image')

        # Проверяем элементы на наличие таблиц
        if isinstance(element, LTRect):
            # Если первый прямоугольный элемент
            if first_element == True and (table_num+1) <= len(tables):
                # Находим ограничивающий прямоугольник таблицы
                lower_side = page.bbox[3] - tables[table_num].bbox[3]
                upper_side = element.y1 
                # Извлекаем информацию из таблицы
                table = extract_table(pdf_path, pagenum, table_num)
                # Преобразуем информацию таблицы в формат структурированной строки
                table_string = table_converter(table)
                # Добавляем строку таблицы в список
                text_from_tables.append(table_string)
                page_content.append(table_string)
                # Устанавливаем флаг True, чтобы избежать повторения содержимого
                table_extraction_flag = True
                # Преобразуем в другой элемент
                first_element = False
                # Добавляем условное обозначение в списки текста и формата
                page_text.append('table')
                line_format.append('table')

            # Проверяем, извлекли ли мы уже таблицы из этой страницы
            # if element.y0 >= lower_side and element.y1 <= upper_side:
            #     pass
            if i + 1 < len(page_elements):
              if not isinstance(page_elements[i+1][1], LTRect):
               table_extraction_flag = False
               first_element = True
               table_num += 1


    # Создаём ключ для словаря
    dctkey = 'Page_'+str(pagenum)
    # Добавляем список списков как значение ключа страницы
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

# Закрываем объект файла pdf
pdfFileObj.close()

# Удаляем созданные дополнительные файлы
os.remove('cropped_image.pdf')
os.remove('PDF_image.png')

# Удаляем содержимое страницы
result = ''.join(text_per_page['Page_0'][4])

#Весь текст в одну переменную
all_text = ""

for page_content in text_per_page.values():
    page_text = page_content[4]  # Получаем содержимое страницы
    all_text += ' '.join(page_text) + '\n'  # Добавляем текст страницы к all_text с добавлением символа новой строки

# Выводим результат
print(all_text)

 
  
  
  
  
 Инструкция  
по эксплуатации 
"1С:ПРЕДПРИЯТИЕ 8" 
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
 Москва 
Фирма «1С» 
2020 

ПРАВО ТИРАЖИРОВАНИЯ  
ПРОГРАММНЫХ СРЕДСТВ И ДОКУМЕНТАЦИИ 
ПРИНАДЛЕЖИТ ФИРМЕ «1С» 
Приобретая систему «1С:Предприятие»,  
вы тем самым даете согласие не допускать 
копирования программ и документации 
без письменного разрешения фирмы «1С» 
 © ООО «1С», 2020 
Фирма «1С», Москва, 123056, а/я 64. 
Отдел продаж: Селезневская ул., 21, 
телефон: (495) 737-92-57, 
факс: (495) 681-44-07. 
E-mail: 1c@1c.ru 
URL: www.1c.ru, www.v8.1c.ru 
Группа разработки программ – А. Алексеев, А. Безбородов, Д. Бескоровайнов, 
П. Василец,  А. Виноградов,  А. Волков,  И. Гольштейн,  Е. Горностаев, 
Г. Дамье,  А. Даровских,  О. Дерут,  Н. Евграфов,  Б. Евтифеев,  Д. Зарецкий, 
Д. Ивашов,  С. Копиенко,  Н. Корсаков,  С. Кравченко,  В.  Кудрявцев, 
П. Кукушкин, А. Лакутин, М. Лейбович, Г. Леонтьев, А. Лехан, А. Макеев, 
А. Медведев,  Е. Митрошкин,  С. Мурзин,  С. Нуралие

Получение заголовков из содержания

In [50]:
print(text_per_page['Page_2'][4])

['Оглавление \n', 'Введение \n', '11 \n', 'Структура Руководства ........................................................................... 11 \nЧто вы должны знать .............................................................................. 13 \nСопроводительные \n', 'комплекта \n', 'поставки \n', 'файлы \n', '«1С:Предприятия 8.3» .............................................................. 14 \n', 'Глава 1. \nГлава 2. \n', 'О системе..................................................................... 16 \nНачало работы ......................................................... 17 \n2.1.  Запуск приложения ...................................................................... 17 \n2.2.  Вход в систему .............................................................................. 17 \n2.2.1.  Запуск веб-клиента ................................................................... 18 \n2.2.2.  Установка скорости соединения ............................................. 18 \n2.3.  Доба

In [96]:
import re

# regex = r'(?:Глава\s+(\d+)\.\s+)?(.+?)\s+(\d+)'
pattern = re.compile(r'(\d+(\.\d+)*)?\s*(.*?)\s+(\d+)\s*\n')

matches = pattern.findall(''.join(text_per_page['Page_3'][4]))
# matches = re.finditer(regex, ''.join(text_per_page['Page_2'][4]))

content_dict = {}

for match in matches:
    chapter_number = match[0] if match[0] else 'None'
    title = match[2].strip().replace('.', '')
    page_number = match[3]
    # print(f"{chapter_number}{title} {page_number}")
    content_dict[page_number] = {'Заголовок': chapter_number.replace(' ', '') + '. ' + title.replace(' ', ' ')}


In [97]:
titles = []
for page, info in content_dict.items():
    print(f"Страница: {page},  {info['Заголовок']}")
    titles.append(info['Заголовок'])

Страница: 38,  4.8.   Таблица 
Страница: 41,  4.9.1.   Командная панель 
Страница: 42,  4.12.   Поле HTML-документа 
Страница: 43,  4.13.   Поле графической схемы 
Страница: 44,  4.14.   Поле картинки 
Страница: 45,  4.15.   Поле табличного документа 
Страница: 47,  4.16.   Поле форматированного документа 
Страница: 49,  4.17.   Операции перетаскивания 
Страница: 51,  4.18.   Работа формы с сохранением настроек 
Страница: 52,  4.19.   Особенности некоторых форм 
Страница: 53,  5.1.   Просмотр списка 
Страница: 54,  5.1.1.   Иерархические списки 
Страница: 57,  5.2.1.   Создание группы списка 
Страница: 58,  5.3.   Редактирование элемента списка 
Страница: 59,  5.3.2.   Выбор значения из списка 
Страница: 61,  5.3.3.   Поиск в списках
Страница: 62,  5.4.   Настройка списка 
Страница: 63,  5.4.1.   Отбор 
Страница: 64,  5.4.3.   Группировка 
Страница: 65,  5.4.5.   Настройка периода 
Страница: 68,  5.5.   Печать списка 
Страница: 69,  6.2.   Ввод элемента на основании 
Страница: 70,  6.3

In [98]:
print(all_text[60000:])

сервисные команды и 
др. 
 СОВЕТ.  При  работе  с  клавиатуры  для  перехода  в  главное  меню 
текущего окна следует нажать клавишу F10. 
 В центре располагается заголовок приложения. 
В  правой  части  области  системных  команд  по  умолчанию 
расположены  команды  работы  с  файлами,  работы  со  ссылками  и 
стандартные функции, такие как вызов календаря, калькулятора и т. д. 
Закрытие основного  окна и выход из программы осуществляются при 
вызове  команды  Файл –  Выход  главного  меню  или  при  нажатии 
кнопки  Закрыть  основного  окна.  Поддерживается  возможность 
навигации по программе при помощи клавиатуры. Таблицы сочетаний 
клавиш для навигации содержатся во встроенной справке. 
 3.1.8. Меню функций 
Меню  функций  предоставляет  удобный  доступ  к  командам  любого 
раздела  и  содержит  все  команды  текущего  раздела,  сгруппированные 
аналогично составу панелей навигации и действий. 
Вызов  меню  функций  выполняется  нажатием  кнопки,  расположенной 
в  области  сис

In [103]:
import re
from sentence_transformers import SentenceTransformer, util
import numpy as np

c:\Users\Saveliy\Documents\SawaProject\searchPDFDefinitionsAndLearn\opensearch\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [104]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

title_embeddings = model.encode(titles)

c:\Users\Saveliy\Documents\SawaProject\searchPDFDefinitionsAndLearn\opensearch\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Saveliy\Documents\SawaProject\searchPDFDefinitionsAndLearn\opensearch\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [108]:
all_text[400000:401000]

'айл – Сохранить как. \nОбычно  табличные  документы  сохраняются  в  специальном  формате, \nиспользуемом  системой  «1С:Предприятие»  для  хранения  табличных \nдокументов.  Такие  файлы  имеют  расширение  *.mxl.  Также  имеется \nвозможность  для  сохранения  табличного  документа  в  других \nформатах – лист  Excel  (расширение  *.xls),  документ  HTML,  документ \nHTML3,  документ  HTML4,  документ  HTML5  (расширение  *.htm), \nдокумент  PDF  (расширение  *.pdf),  текстовый  файл  и  текстовый  файл \nUNICODE  (расширение  *.txt),  табличный  документ  v7.х  (формата \nпредыдущих  версий  системы  «1С:Предприятия»),  лист  Excel95 \n(расширение  *.xls),  лист  Excel97  (расширение  *.xls),  документ \nWord2007  (расширение  *.docx),  лист  Excel2007  (расширение  *.xlsx), \nэлектронная  таблица  ODF  (расширение  *.ods).  Для  этого  нужно \nвыбрать в поле Сохранить как: необходимый формат файла. \n следующие \n ПРИМЕЧАНИЕ  1.  Необходимо  иметь  в  виду \nограничения, существую

In [111]:

# Регулярное выражение для разделения текста по заголовкам
pattern = re.compile(r'(\d{1,2}\.\d{1,2}\. [^\n]+|О системе|Начало работы)')

# Инициализация словаря для хранения результатов
paragraphs_dict = {title: "" for title in titles}

# Текущий заголовок (инициализируется пустой строкой)
current_title = ""

# Разделение текста на строки
lines = all_text[400000:401000]

# Проход по строкам и заполнение словаря
for line in sentences:
    line = line.strip()
    if not line:
        continue
    
    match = pattern.match(line)
    if match:
        current_title = match.group(0)
    else:
        # Векторизация строки и поиск наиболее похожего заголовка
        line_embedding = model.encode([line])[0]
        similarities = util.pytorch_cos_sim(line_embedding, title_embeddings)[0].numpy()
        most_similar_idx = np.argmax(similarities)
        
        if similarities[most_similar_idx] > 0.7:  # Порог сходства
            current_title = titles[most_similar_idx]
        print(line, similarities)
    # if current_title:
    #     paragraphs_dict[current_title] += line + ' '
    

# Удаление лишних пробелов
for title in paragraphs_dict:
    paragraphs_dict[title] = paragraphs_dict[title].strip()

# Вывод результатов
for title, paragraph in paragraphs_dict.items():
    print(f"Заголовок: {title}\nАбзац: {paragraph}\n")

Инструкция  
по эксплуатации 
"1С:ПРЕДПРИЯТИЕ 8" 
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
  
 Москва 
Фирма «1С» 
2020 

ПРАВО ТИРАЖИРОВАНИЯ  
ПРОГРАММНЫХ СРЕДСТВ И ДОКУМЕНТАЦИИ 
ПРИНАДЛЕЖИТ ФИРМЕ «1С» 
Приобретая систему «1С:Предприятие»,  
вы тем самым даете согласие не допускать 
копирования программ и документации 
без письменного разрешения фирмы «1С» 
 © ООО «1С», 2020 
Фирма «1С», Москва, 123056, а/я 64.
Отдел продаж: Селезневская ул., 21, 
телефон: (495) 737-92-57, 
факс: (495) 681-44-07.
E-mail: 1c@1c.ru 
URL: www.1c.ru, www.v8.1c.ru 
Группа разработки программ – А. Алексеев, А. Безбородов, Д. Бескоровайнов, 
П. Василец,  А. Виноградов,  А. Волков,  И. Гольштейн,  Е. Горностаев, 
Г. Дамье,  А. Даровских,  О. Дерут,  Н. Евграфов,  Б. Евтифеев,  Д. Зарецкий, 
Д. Ивашов,  С. Копиенко,  Н. Корсаков,  С. Кравченко,  В.  Кудрявцев, 
П. Кукушкин, А. Лакутин, М. Лейбович, Г. Леонтьев, А. Лехан, А. Макеев, 
А. Медведев,  Е. Митрошкин,  С. Мурзин,  С. Нуралиев,  Л. Онучин, 
М

KeyboardInterrupt: 

In [101]:
import nltk
nltk.download('punkt')

from nltk.tokenize import sent_tokenize
sentences=sent_tokenize(all_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Saveliy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
print(sentences)

[' \n  \n  \n  \n  \n Инструкция  \nпо эксплуатации \n"1С:ПРЕДПРИЯТИЕ 8" \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n Москва \nФирма «1С» \n2020 \n\nПРАВО ТИРАЖИРОВАНИЯ  \nПРОГРАММНЫХ СРЕДСТВ И ДОКУМЕНТАЦИИ \nПРИНАДЛЕЖИТ ФИРМЕ «1С» \nПриобретая систему «1С:Предприятие»,  \nвы тем самым даете согласие не допускать \nкопирования программ и документации \nбез письменного разрешения фирмы «1С» \n © ООО «1С», 2020 \nФирма «1С», Москва, 123056, а/я 64.', 'Отдел продаж: Селезневская ул., 21, \nтелефон: (495) 737-92-57, \nфакс: (495) 681-44-07.', 'E-mail: 1c@1c.ru \nURL: www.1c.ru, www.v8.1c.ru \nГруппа разработки программ – А. Алексеев, А. Безбородов, Д. Бескоровайнов, \nП. Василец,  А. Виноградов,  А. Волков,  И. Гольштейн,  Е. Горностаев, \nГ. Дамье,  А. Даровских,  О. Дерут,  Н. Евграфов,  Б. Евтифеев,  Д. Зарецкий, \nД. Ивашов,  С. Копиенко,  Н. Корсаков,  С. Кравченко,  В.  Кудрявцев, \nП. Кукушкин, А. Лакутин, М. Лейбович, Г. Леонтьев, А. Лехан, А. Макеев, 